### Day One

In [1]:
# Input

with open("day_one_input.txt") as f:
    data = [int(i) for i in f.read().split("\n")]
print(data[:10])

[159, 170, 171, 170, 168, 167, 166, 164, 163, 154]


In [2]:
# Part One

len([True for i in range(len(data)-1) if data[i+1] > data[i]])

1451

In [3]:
# Part Two

len([True for i in range(len(data)) if sum(data[i+1:i+4]) > sum(data[i:i+3])])

1395

### Day Two

In [4]:
# Input

with open("day_two_input.txt") as f:
    data = f.read().split("\n")
print(data[:10])

['forward 4', 'down 9', 'forward 2', 'forward 2', 'down 7', 'up 2', 'down 9', 'up 8', 'down 7', 'down 9']


In [5]:
# Parse data

instructions = [(line.split()[0], int(line.split()[1])) for line in data]

In [6]:
# Part One

horizontal = sum(dist for instr, dist in instructions if instr == "forward")
depth = sum(-dist if instr == "up" else dist if instr == "down" else 0 for instr, dist in instructions)
horizontal*depth

1893605

In [7]:
# Part Two

aim, horizontal, depth = 0, 0, 0
for instr, dist in instructions:
    aim += -dist if instr == "up" else dist if instr == "down" else 0
    if instr == "forward":
        horizontal += dist
        depth += dist*aim
horizontal*depth

2120734350

### Day Three

In [8]:
# Input

with open("day_three_input.txt") as f:
    data = f.read().split("\n")
print(data[:10])

['111111010011', '110011001100', '010011111000', '101001100011', '011011100110', '110100011011', '001000001010', '011011101000', '110011001000', '001011101010']


In [9]:
# Part One

from collections import Counter

gamma, epsilon = [], []
for bit in range(len(data[0])):
    counts = Counter([diagnostic[bit] for diagnostic in data]).most_common()
    gamma.append(counts[0][0])
    epsilon.append(counts[-1][0])
int("".join(gamma), 2)*int("".join(epsilon), 2)

3985686

In [10]:
# Part Two

ratings = []
for counter_index, bit_value in zip((0, -1), ("1", "0")):
    data_subset = data
    bit_index = 0
    while len(data_subset) > 1:
        counts = Counter([diagnostic[bit_index] for diagnostic in data_subset]).most_common()
        bit_test = bit_value if counts[0][1] == counts[1][1] else counts[counter_index][0]
        data_subset = [diagnostic for diagnostic in data_subset if diagnostic[bit_index] == bit_test]
        bit_index += 1
    ratings.append(int("".join(data_subset[0]), 2))
ratings[0]*ratings[1]

2555739

### Day Four

In [11]:
# Input

import numpy as np

with open("day_four_input.txt") as f:
    numbers = f.readline().strip()

numbers = [int(n) for n in numbers.split(",")]
boards = np.recfromtxt("day_four_input.txt", skip_header=1).reshape(100, 5, 5)

print(numbers[:10])
print(boards[0])

[49, 48, 98, 84, 71, 59, 37, 36, 6, 21]
[[86 46 47 61 57]
 [44 74 17  5 87]
 [78  8 54 55 97]
 [11 90  7 75 70]
 [81 50 84 10 60]]


In [12]:
# Part One

results = []
for i, board in enumerate(boards):
    mask = np.full(board.shape, False)
    numbers_iter = iter(numbers)
    while 5 not in np.sum(mask, axis=0) and 5 not in np.sum(mask, axis=1):
        last_called = next(numbers_iter)
        mask = mask | (board == last_called)
    results.append({
        "score": int(last_called*board[~mask].sum()),
        "order": numbers.index(last_called)
    })

sorted(results, key=lambda k: k["order"])[0]["score"]

65325

In [13]:
# Part Two

sorted(results, key=lambda k: k["order"], reverse=True)[0]["score"]

4624

### Day Five

In [14]:
# Input

with open("day_five_input.txt") as f:
    data = f.read().split("\n")
data[:10]

['599,531 -> 599,32',
 '435,904 -> 435,489',
 '768,714 -> 768,187',
 '845,552 -> 596,801',
 '167,680 -> 167,445',
 '45,887 -> 45,346',
 '780,295 -> 179,896',
 '310,539 -> 602,831',
 '535,556 -> 349,556',
 '797,180 -> 797,62']

In [15]:
# Part One

from shapely import wkt
from rasterio.features import rasterize
from rasterio.enums import MergeAlg

vents = [wkt.loads(
    f"LINESTRING ({line.replace(',', ' ').replace(' -> ', ', ')})"
) for line in data]
orthogonal_vents = [vent for vent in vents if len(set(vent.bounds)) < 4]
np.sum(rasterize(orthogonal_vents, out_shape=(1000,1000), merge_alg=MergeAlg("ADD")) >=2)

7644

In [16]:
# Part Two

np.sum(rasterize(vents, out_shape=(1000,1000), merge_alg=MergeAlg("ADD")) >=2)

18627

### Day Six

In [17]:
# Input

with open("day_six_input.txt") as f:
    data = f.read().split(",")
data[:10]

['3', '5', '4', '1', '2', '1', '5', '5', '1', '1']

In [18]:
%%time

# Part One

timers = [int(d) for d in data]

for day in range(80):
    new_timers = []
    for i, timer in enumerate(timers):
        timers[i] = timer-1 if timer-1 >= 0 else 6
        new_timers.append(8) if timer-1 == -1 else None
    timers.extend(new_timers)
    
len(timers)

CPU times: user 1.54 s, sys: 4.32 ms, total: 1.54 s
Wall time: 1.55 s


374927

In [19]:
%%time

# Part Two -- refactor of Part One

from collections import deque

timers = deque([0]*9)
for d in [int(d) for d in data]:
    timers[d] += 1
    
for day in range(256):
    timers.rotate(-1)
    timers[6] += timers[8]

sum(timers)

CPU times: user 1.21 ms, sys: 5 µs, total: 1.22 ms
Wall time: 1.23 ms


1687617803407

### Day Seven

In [20]:
# Input

with open("day_seven_input.txt") as f:
    data = f.read().split(",")
data[:10]

['1101', '1', '29', '67', '1102', '0', '1', '65', '1008', '65']

In [21]:
# Part One

from statistics import median

positions = [int(d) for d in data]
optimum = int(median(positions))
fuel = [abs(i-optimum) for i in positions]
sum(fuel)

341558

In [22]:
%%time

# Part Two

optimums = []

for o in range(min(positions), max(positions)):
    fuel = 0
    for p in positions:
        if abs(p-o) > 0:
            fuel += max(np.cumsum(np.arange(abs(p-o)+1)))
    optimums.append({"position":o, "fuel": fuel})

sorted(optimums, key=lambda o: o["fuel"])[0]

CPU times: user 2min 5s, sys: 195 ms, total: 2min 5s
Wall time: 2min 5s


{'position': 484, 'fuel': 93214037}